In [1]:
# 引入数据处理包
import numpy as np
import pandas as pd

In [2]:
# 引入数据
# 附件表单1数据
types_pre = {"文物编号":str}
data_one = pd.read_excel("附件.xlsx",sheet_name="表单1",dtype=types_pre)
data_one.head(2)

,文物编号,纹饰,类型,颜色,表面风化
0,01,C,高钾,蓝绿,无风化
1,02,A,铅钡,浅蓝,风化


In [3]:
# 附件表单2数据
data_two = pd.read_excel("附件.xlsx",sheet_name="表单2")
data_two.head(2)

,文物采样点,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,01,69.33,NaN,9.99,6.32,0.87,3.93,1.74,3.87,NaN,NaN,1.17,NaN,NaN,0.39
1,02,36.28,NaN,1.05,2.34,1.18,5.73,1.86,0.26,47.43,NaN,3.57,0.19,NaN,NaN


In [4]:
# columns准备
data_one_list = data_one.columns.to_list()
data_two_list = data_two.columns.to_list()
data_list = data_two_list + data_one_list

In [5]:
# 用于保存和并后的结果
data_merge = pd.DataFrame(columns=data_list)

In [6]:
data_two.iloc[0, :].values
data_one.iloc[0, :].values
np.hstack((data_two.iloc[0, :].values, data_one.iloc[0, :].values))

array(['01', 69.33, nan, 9.99, 6.32, 0.87, 3.93, 1.74, 3.87, nan, nan,
       1.17, nan, nan, 0.39, '01', 'C', '高钾', '蓝绿', '无风化'], dtype=object)

In [7]:
# 将两个表合并
for i in range(data_two.shape[0]):
    for j in range(data_one.shape[0]):
        # print(data_two.iloc[i, 0])
        if data_one.iloc[j, 0] in data_two.iloc[i, 0]:
            v_tmp_one = data_one.iloc[j, :].values
            v_tmp_two = data_two.iloc[i, :].values
            data_merge.loc[i] = np.hstack((v_tmp_two, v_tmp_one))
            break

In [21]:
print("data_merge.shape",data_merge.shape)
data_merge.head(2)

data_merge.shape (69, 20)


,文物采样点,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),文物编号,纹饰,类型,颜色,表面风化
0,01,69.33,NaN,9.99,6.32,0.87,3.93,1.74,3.87,NaN,NaN,1.17,NaN,NaN,0.39,01,C,高钾,蓝绿,无风化
1,02,36.28,NaN,1.05,2.34,1.18,5.73,1.86,0.26,47.43,NaN,3.57,0.19,NaN,NaN,02,A,铅钡,浅蓝,风化


In [33]:
# 删除无效数据
sum_a_1 = data_merge.iloc[:,1:data_two.shape[1]].sum(axis=1)
data_res = data_merge[(sum_a_1 >= 85) & (sum_a_1 <= 105)]
print("剔除无效数据后：")
print("data_res.shape",data_res.shape)
data_res.head(2)

剔除无效数据后：
data_res.shape (67, 20)


,文物采样点,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),文物编号,纹饰,类型,颜色,表面风化
0,01,69.33,NaN,9.99,6.32,0.87,3.93,1.74,3.87,NaN,NaN,1.17,NaN,NaN,0.39,01,C,高钾,蓝绿,无风化
1,02,36.28,NaN,1.05,2.34,1.18,5.73,1.86,0.26,47.43,NaN,3.57,0.19,NaN,NaN,02,A,铅钡,浅蓝,风化


In [31]:
# 展示无效数据
data_err = data_merge[(sum_a_1 < 85) | (sum_a_1 > 105)]
data_err_pd = pd.DataFrame(columns=["文物采样点","成分比例累加和"])
data_err_pd["文物采样点"] = data_err.iloc[:,0].values
data_err_pd["成分比例累加和"] = data_err.iloc[:,1:data_two.shape[1]].sum(axis=1).values
data_err_pd.head()

,文物采样点,成分比例累加和
0,15,79.47
1,17,71.89


In [34]:
# 保存处理结果
data_res.to_excel("问题一处理后的数据.xlsx")

In [43]:
# 对缺失数据进行处理
data_fill = data_res.copy()
data_fill.iloc[:,0:data_two.shape[1]] = data_res.iloc[:,0:data_two.shape[1]].fillna(0)
data_fill.iloc[:,-data_one.shape[1]:] = data_res.iloc[:,-data_one.shape[1]:].fillna("missing")
data_fill.head(2)

,文物采样点,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),文物编号,纹饰,类型,颜色,表面风化
0,01,69.33,0.0,9.99,6.32,0.87,3.93,1.74,3.87,0.00,0.0,1.17,0.00,0.0,0.39,01,C,高钾,蓝绿,无风化
1,02,36.28,0.0,1.05,2.34,1.18,5.73,1.86,0.26,47.43,0.0,3.57,0.19,0.0,0.00,02,A,铅钡,浅蓝,风化


In [45]:
data_fill.to_excel("问题一填补后.xlsx")